In [37]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
!pip install gdown

In [ ]:
!pip install torch-summary

In [ ]:
import gdown
gdown.download("https://drive.google.com/file/d/1OIUMav8Mx1teLsaOXEtuT4l7rlfjX_NV/view?usp=sharing", fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1OIUMav8Mx1teLsaOXEtuT4l7rlfjX_NV
From (redirected): https://drive.google.com/uc?id=1OIUMav8Mx1teLsaOXEtuT4l7rlfjX_NV&confirm=t&uuid=3fd96b05-9890-4cf5-b182-6f5b206b2ed9
To: /content/ctdata.zip
100%|██████████| 5.46G/5.46G [01:22<00:00, 66.0MB/s]


'ctdata.zip'

In [ ]:
!unzip -qo ctdata.zip

In [54]:
import torch
from rich import print
import random
# from utils import load_video
import glob
import numpy as np
import gc
from torchsummary import summary

In [68]:
files = glob.glob("CourtTrial/Clips/Identities/*/*/*/*/*aligned/video.npy")
random.shuffle(files)
device = torch.device("cpu")

In [69]:
class VideoDataset(torch.utils.data.Dataset):
    def __init__(self, files):
        self.files = files
    def __len__(self):
        return len(self.files)
    def __getitem__(self, idx):
        # print("[red]Loading video", idx, "...")
        # load the video as an np array
        video_path = self.files[idx]
        video = np.load(video_path)
        video = video / 255.0
        # pad the video to the max number of frames
        video = torch.tensor(video, dtype=torch.float32)
        # change ordering
        video = video.permute(3, 0, 1, 2)

        y = 0 if "truth" in video_path else 1
        return video, y
    def truth_lies(self):
        lies = 0
        truths = 0
        for file in self.files:
            if "truth" in file:
                truths += 1
            elif "lie" in file:
                lies += 1
        return truths, lies

In [70]:
class CNNModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = torch.nn.Sequential(
            torch.nn.Conv3d(3, 32, kernel_size=(2,2,2)),
            torch.nn.ReLU(),

            torch.nn.Conv3d(32, 64, kernel_size=(2,2,2)),
            torch.nn.MaxPool3d((2,2,2)),
            torch.nn.ReLU(),

            torch.nn.Conv3d(64, 32, kernel_size=(2,2,2)),
            torch.nn.MaxPool3d((4, 4, 4)),
            torch.nn.ReLU(),

            torch.nn.Flatten(),

            torch.nn.Linear(21888, 5000),
            torch.nn.ReLU(),

            torch.nn.Linear(5000, 500),
            torch.nn.ReLU(),

            torch.nn.Linear(500, 2),
            torch.nn.Softmax(dim=1)
        )
    def forward(self, x):
        x = self.cnn(x)
        return x


In [71]:
def collater(batch):
    X = []
    y = []
    for x, _y in batch:
        X.append(torch.nn.functional.pad(x, (0, 0, 0, 0, 0, 2443 - x.shape[1])))
        y.append(_y)
    X = torch.stack(X).to(device)
    y = torch.tensor(y).to(device)
    return X, y

In [72]:
print("[red]Loading dataset...")
dataset = VideoDataset(files)
print("[green]Number of videos:", len(dataset))

Loading dataset...

Number of videos: 112

In [73]:
for i in range(5):
    print("[blue]Shape of video", i, ":", dataset[i][0].shape)



Shape of video 0 :
torch.Size([3, 330, 112, 112])

Shape of video 1 :
torch.Size([3, 870, 112, 112])

Shape of video 2 :
torch.Size([3, 173, 112, 112])

Shape of video 3 :
torch.Size([3, 1298, 112, 112])

Shape of video 4 :
torch.Size([3, 230, 112, 112])

In [74]:

print("[red]Making model...")
model = CNNModel()
print(model)
# summary(model, torch.Tensor(3, 2443, 112, 112))

Making model...

CNNModel(
  (cnn): Sequential(
    (0): Conv3d(3, 32, kernel_size=(2, 2, 2), stride=(1, 1, 1))
    (1): ReLU()
    (2): Conv3d(32, 64, kernel_size=(2, 2, 2), stride=(1, 1, 1))
    (3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): ReLU()
    (5): Conv3d(64, 32, kernel_size=(2, 2, 2), stride=(1, 1, 1))
    (6): MaxPool3d(kernel_size=(4, 4, 4), stride=(4, 4, 4), padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=21888, out_features=5000, bias=True)
    (10): ReLU()
    (11): Linear(in_features=5000, out_features=500, bias=True)
    (12): ReLU()
    (13): Linear(in_features=500, out_features=2, bias=True)
    (14): Softmax(dim=1)
  )
)

In [75]:
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()


train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset = VideoDataset(files[:train_size])
test_dataset = VideoDataset(files[train_size:])


print("[red]Training dataset size:", len(train_dataset))
print("[green]Test dataset size:", len(test_dataset))

print("[red]Number of truths and lies in training dataset:", train_dataset.truth_lies())
print("[green]Number of truths and lies in test dataset:", test_dataset.truth_lies())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collater)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=collater)

Training dataset size: 89

Test dataset size: 23

Number of truths and lies in training dataset:
(52, 37)

Number of truths and lies in test dataset:
(8, 15)

In [ ]:
print("[red]Training model...")

for epoch in range(10):
    model.train()
    for i, (X, y) in enumerate(train_loader):
        print("[red]Epoch", epoch, "Iteration", i, "...")
        # print("[green]X:", X.shape)
        # print("Size taken by X", X.element_size() * X.nelement() / 1e6, "MB")
        opt.zero_grad()
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        loss.backward()
        opt.step()
        if i % 10 == 0:
            print(f"Epoch {epoch}, Iteration {i}, Loss {loss.item()}")

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (X, y) in enumerate(test_loader):
            y_pred = model(X)
            print(y_pred)
            correct += (y_pred.argmax(1) == y).sum().item()
            total += 1
        print(f"Epoch {epoch}, Test Accuracy: {correct / total}")

Training model...

Epoch 0 Iteration 0 ...

In [ ]:
!nvidia-smi

In [ ]:
!sudo kill -9 2718

kill: (2718): No such process


In [ ]:
!nvidia-smi

Thu Jun 20 05:25:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0              31W /  70W |  11207MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!lsof /dev/nvidia*

/bin/bash: lsof: command not found
